In [ ]:
!nvidia-smi

In [ ]:
!pip install -qq torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -qq mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html
!git clone https://github.com/open-mmlab/mmdetection
%cd mmdetection

!pip install -e .

In [ ]:
!pip install -q --upgrade wandb

# Imports

In [ ]:
import wandb
import os
import shutil
import sys
import json
import torch,torchvision
from collections import defaultdict
from datetime import datetime
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
from mmcv import Config

import mmdet
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.apis import init_detector, inference_detector


import random
import numpy as np
from pathlib import Path

In [ ]:
seed = 42
set_random_seed(seed, deterministic=False)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ['PYTHONHASHSEED'] = str(seed)

In [ ]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
wandb_api = user_secrets.get_secret("wandb-key") 
wandb.login(key=wandb_api)

# Constants

In [ ]:
PROJECT_NAME = 'Final-Covid19-Detection'
NOTEBOOK_NAME = 'TFM - Detection Model Testing'
EXP_NAME = 'DETR_pretrained_refinement'


In [ ]:
model_name = f'deformable_detr/deformable_detr_refine_r50_16x2_50e_coco'

# Wandb Initiation and file downloading

In [ ]:
import pytz
timezone = pytz.timezone('Europe/Madrid')
now = datetime.now(timezone)

dt_string = now.strftime("%d/%m_%H:%M")

In [ ]:
run = wandb.init(project=PROJECT_NAME,
                 name=f'Downloading_files_{EXP_NAME}_{dt_string}',
                job_type='file_transfer',
                notes=f'Notebook: {NOTEBOOK_NAME}')


In [ ]:
artifact = run.use_artifact('alvaromoureupm/Final-Covid19-Detection/512x512_train_val_coco_files:v3')
artifact_dir = artifact.download()
run.finish()

In [ ]:
for file in os.listdir(artifact_dir):
    shutil.copyfile(os.path.join(artifact_dir,file),f'/kaggle/working/{file}')

In [ ]:
train_ann_file = '/kaggle/working/coco_train_512x512.json'
val_ann_file = '/kaggle/working/coco_val_512x512.json'

# Example Model Training

### Training Hyperparameters

In [ ]:
baseline_cfg_path = f'/kaggle/working/mmdetection/configs/{model_name}.py'
cfg = Config.fromfile(baseline_cfg_path)
job_folder = f'/kaggle/working/{model_name}'
if not os.path.exists(job_folder):
    os.makedirs(job_folder)
cfg.work_dir = job_folder
cfg.seed = 42
cfg.model.bbox_head.num_classes = 1
cfg.gpu_ids = [0]
cfg.runner.max_epochs = 20
cfg.total_epochs = 20
cfg.load_from = 'https://download.openmmlab.com/mmdetection/v2.0/deformable_detr/deformable_detr_refine_r50_16x2_50e_coco/deformable_detr_refine_r50_16x2_50e_coco_20210419_220503-5f5dff21.pth'

In [ ]:
print(cfg.pretty_text)

## Configuring Dataset Type

### Train/Val Split (images without bounding boxes also)

In [ ]:
cfg.dataset_type = 'CocoDataset'
cfg.classes= ('Covid_Opacity',)
cfg.data.train.img_prefix = '/kaggle/input/siim-covid19-images-metadata-256-512-768/images_metadata_256_512_768/train_512x512'
cfg.data.train.classes = cfg.classes
cfg.data.train.ann_file = train_ann_file
cfg.data.train.type = 'CocoDataset'

cfg.data.val.img_prefix = '/kaggle/input/siim-covid19-images-metadata-256-512-768/images_metadata_256_512_768/train_512x512'
cfg.data.val.classes = cfg.classes
cfg.data.val.ann_file = val_ann_file
cfg.data.train.type = 'CocoDataset'

cfg.data.test.img_prefix = '/kaggle/input/siim-covid19-images-metadata-256-512-768/images_metadata_256_512_768/test_512x512'
cfg.data.test.classes = cfg.classes
cfg.data.test.ann_file = val_ann_file
cfg.data.test.type = 'CocoDataset'

In [ ]:
cfg.data.workers_per_gpu = 2

In [ ]:
cfg.log_config.hooks = [dict(type='TextLoggerHook'),
                        dict(type='WandbLoggerHook',
                             init_kwargs=dict(project=PROJECT_NAME,
                                              name=f'{EXP_NAME}_{dt_string}',
                                              job_type='model_train',
                                              entity='alvaromoureupm',
                                             notes=f'Notebook: {NOTEBOOK_NAME}',
                                             config=cfg._cfg_dict.to_dict()),
                             log_artifact=True,
                             interval = 100
                            )
                       ]

In [ ]:
print(cfg.pretty_text)

### Save Config File

In [ ]:
cfg_path = f'{job_folder}_{EXP_NAME}.py'
print(cfg_path)

# Save config file for inference later
cfg.dump(cfg_path)
#print(cfg.pretty_text)

meta = dict()
meta['exp_name'] = os.path.basename(cfg_path)
cfg.dump(os.path.join(cfg.work_dir, meta['exp_name']))

In [ ]:
model = build_detector(cfg.model,
                       train_cfg=cfg.get('train_cfg'),
                       test_cfg=cfg.get('test_cfg'))
model.init_weights()

In [ ]:
cfg.device = 'cuda'
datasets = [build_dataset(cfg.data.train)]

In [ ]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

### Loging the best model in Wandb

In [ ]:
log_file = f'{job_folder}/None.log.json'

# Source: mmdetection/tools/analysis_tools/analyze_logs.py 
def load_json_logs(json_logs):
    # load and convert json_logs to log_dict, key is epoch, value is a sub dict
    # keys of sub dict is different metrics, e.g. memory, bbox_mAP
    # value of sub dict is a list of corresponding values of all iterations
    log_dicts = [dict() for _ in json_logs]
    for json_log, log_dict in zip(json_logs, log_dicts):
        with open(json_log, 'r') as log_file:
            for line in log_file:
                log = json.loads(line.strip())
                # skip lines without `epoch` field
                if 'epoch' not in log:
                    continue
                epoch = log.pop('epoch')
                if epoch not in log_dict:
                    log_dict[epoch] = defaultdict(list)
                for k, v in log.items():
                    log_dict[epoch][k].append(v)
    return log_dicts

log_dict = load_json_logs([log_file])
best_epoch = np.argmax([item['bbox_mAP'][0] for item in log_dict[0].values()])+1
best_epoch

In [ ]:
run = wandb.init(project=PROJECT_NAME,
                 name=f'Saving_model{EXP_NAME}_{dt_string}',
                job_type='model_save',
                notes=f'Notebook: {NOTEBOOK_NAME}')

artifact = wandb.Artifact(EXP_NAME, type='model')

artifact.add_file(f'{job_folder}/epoch_{best_epoch}.pth')
artifact.add_file(cfg_path)

run.log_artifact(artifact)
run.finish()